## Please run this notebook in colab or any GPU backend system

In [ ]:
%cd /content
!mkdir traning_demo
!mkdir traning_demo/pre-trained-models
!mkdir traning_demo/annotations
!mkdir traning_demo/export_model

In [ ]:
!pip install tensorflow-gpu

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd /content/models/research

In [6]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

In [ ]:
%cd cocoapi/PythonAPI

In [ ]:
!make

In [10]:
%cp -r pycocotools /content/models/research

In [ ]:
%pwd

In [ ]:
%cd ../..

In [13]:
%cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
%cd /content/traning_demo/pre-trained-models

In [17]:
# !rm -r efficientdet_d0_coco17_tpu-32

In [ ]:
# https://drive.google.com/file/d/1C8J5g7zRr3rfbr9db5Q8HCOfXglOG7Kb/view?usp=sharing
# Efficientdet pretrained model with wrinkles and darkspots
# Comment this line if you want try with custom one

!gdown --id 1C8J5g7zRr3rfbr9db5Q8HCOfXglOG7Kb
!unzip efdnet_2C.zip
!mv /content/traning_demo/pre-trained-models/content/traning_demo/export_model /content/traning_demo/pre-trained-models
!mv /content/traning_demo/pre-trained-models/export_model darkspots




# https://drive.google.com/file/d/1-0zzdLi3bGk8zlGkAyBhQQPtf_bIuMep/view?usp=sharing
# Wrinkles
!gdown --id 1-0zzdLi3bGk8zlGkAyBhQQPtf_bIuMep
!unzip efdnet.zip
!mv /content/traning_demo/pre-trained-models/content/traning_demo/export_model /content/traning_demo/pre-trained-models
!mv /content/traning_demo/pre-trained-models/export_model Wrinkles




# https://drive.google.com/file/d/1zvHrTZBTLJVsGixF7us2ByPmiGgb3IOj/view?usp=sharing
# Darkspots
# !gdown --id 1zvHrTZBTLJVsGixF7us2ByPmiGgb3IOj
# !unzip darkspots.zip
# !mv /content/traning_demo/pre-trained-models/content/fine_tuned_model /content/traning_demo/pre-trained-models
# !mv /content/traning_demo/pre-trained-models/fine_tuned_model /content/traning_demo/pre-trained-models/darkspots


# Uncomment this if you want to train from scratch
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
# !tar -xf /content/traning_demo/pre-trained-models/efficientdet_d0_coco17_tpu-32.tar.gz

In [ ]:
%cd ..

In [ ]:
%ls

In [22]:
# rm -r /content/traning_demo/dataset
!rm -r /content/traning_demo/export_model/
!mkdir export_model

In [23]:
# https://drive.google.com/file/d/1m3gm3tKGhTQ9a_i_cLD_SS005XMX7ZM_/view?usp=sharing
# Download the dataset

# !gdown --id 1m3gm3tKGhTQ9a_i_cLD_SS005XMX7ZM_
# !unzip /content/traning_demo/dataset.zip

In [ ]:
# https://drive.google.com/file/d/1C1vNYIgXbxhN1p4NgpKX_BU15DHGOQyw/view?usp=sharing
# Download required python files
!gdown --id 1C1vNYIgXbxhN1p4NgpKX_BU15DHGOQyw
!unzip python_files.zip
!mv label_map.pbtxt /content/traning_demo/annotations

In [25]:
# Create train data:
# !python /content/traning_demo/generate_tfrecord.py -x /content/traning_demo/dataset/train -l /content/traning_demo/annotations/label_map.pbtxt -o /content/traning_demo/annotations/train.record

# Create test data:
# !python /content/traning_demo/generate_tfrecord.py -x /content/traning_demo/dataset/test -l /content/traning_demo/annotations/label_map.pbtxt -o/content/traning_demo/annotations/test.record

In [26]:
!mkdir my_model

In [ ]:
# !cp /content/traning_demo/pre-trained-models/efficientdet_d0_coco17_tpu-32/pipeline.config /content/traning_demo/my_model

In [ ]:
%cd /content/traning_demo

In [ ]:
%pwd

In [ ]:
# !rm -r /content/traning_demo/export_model
# !mkdir export_model

In [ ]:
# Train 
# %%time
# !python /content/traning_demo/model_main_tf2.py --model_dir=/content/traning_demo/my_model --pipeline_config_path=/content/traning_demo/my_model/pipeline.config

In [ ]:
# Exporting model
# %%time
# !python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/traning_demo/my_model/pipeline.config --trained_checkpoint_dir /content/traning_demo/my_model --output_directory=/content/traning_demo/export_model

In [ ]:
# !cp -r /content/traning_demo/pre-trained-models/efficientdet_d0_coco17_tpu-32/. /content/traning_demo/export_model

In [ ]:
%%time

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pathlib
import tensorflow as tf
import cv2
import argparse
from google.colab.patches import cv2_imshow

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)



# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR_WRINKLES = '/content/traning_demo/pre-trained-models/Wrinkles'

MIN_CONF_THRESH = float(0.10)

# LOAD THE MODEL

PATH_TO_SAVED_MODEL_WRINKLES = PATH_TO_MODEL_DIR_WRINKLES + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn_wrinkles = tf.saved_model.load(PATH_TO_SAVED_MODEL_WRINKLES)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


In [ ]:
PATH_TO_MODEL_DIR_DARKSPOTS = '/content/traning_demo/pre-trained-models/darkspots'

MIN_CONF_THRESH = float(0.10)

# LOAD THE MODEL

PATH_TO_SAVED_MODEL_DARKSPOTS = PATH_TO_MODEL_DIR_DARKSPOTS + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn_darkspots = tf.saved_model.load(PATH_TO_SAVED_MODEL_DARKSPOTS)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
# import numpy as np
# from PIL import Image
# import matplotlib.pyplot as plt
# import warnings
# import cv2
# warnings.filterwarnings('ignore')


# # PROVIDE PATH TO IMAGE DIRECTORY
# IMAGE_PATHS = '/content/traning_demo/dataset/test/5f4278a310117792c164ed4d3e20f586.jpg'

# # PROVIDE PATH TO LABEL MAP
# PATH_TO_LABELS = '/content/traning_demo/annotations/label_map.pbtxt'

# # LOAD LABEL MAP DATA FOR PLOTTING

# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
#                                                                     use_display_name=True)


# print('Running inference for {}... '.format(IMAGE_PATHS), end='')

# image = cv2.imread(IMAGE_PATHS)
# image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image_expanded = np.expand_dims(image_rgb, axis=0)


# input_tensor = tf.convert_to_tensor(image)

# input_tensor = input_tensor[tf.newaxis, ...]


# detections = detect_fn(input_tensor)

# num_detections = int(detections.pop('num_detections'))
# detections = {key: value[0, :num_detections].numpy()
#                for key, value in detections.items()}
# detections['num_detections'] = num_detections

# detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# image_with_detections = image.copy()

# viz_utils.visualize_boxes_and_labels_on_image_array(
#       image_with_detections,
#       detections['detection_boxes'],
#       detections['detection_classes'],
#       detections['detection_scores'],
#       category_index,
#       use_normalized_coordinates=True,
#       max_boxes_to_draw=200,
#       min_score_thresh=0.4,
#       agnostic_mode=False)

# print('Done')
# # DISPLAYS OUTPUT IMAGE
# cv2_imshow(image_with_detections)
# # cv2.imwrite("Test.jpg", image_with_detections)
# # CLOSES WINDOW ONCE KEY IS PRESSED

In [ ]:
!pip install flask-ngrok

In [ ]:
%cd /content/

In [ ]:
%pwd

In [ ]:
# https://drive.google.com/file/d/1iWvskjmJD4Xr1x2NtV0TYJm116lh1MHq/view?usp=sharing
!gdown --id 1iWvskjmJD4Xr1x2NtV0TYJm116lh1MHq
!unzip webpage.zip

In [ ]:
# https://drive.google.com/file/d/1fMsK5T1S0rkzzth5wVcMHDbY4H5gYZTA/view?usp=sharing
# Efficientnet model. For pluffy eyes
!gdown --id 1fMsK5T1S0rkzzth5wVcMHDbY4H5gYZTA

In [ ]:
# https://drive.google.com/file/d/1OzT-szWYdMFQIVX-KQ7PMLhjuSdgAf7Z/view?usp=sharing
# !gdown --id 1OzT-szWYdMFQIVX-KQ7PMLhjuSdgAf7Z

In [39]:
import os
import urllib.request
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename


from keras.models import load_model
import numpy as np
import cv2
from PIL import Image
from numpy import asarray


from flask_ngrok import run_with_ngrok


import sys
import os
import dlib
import glob
from skimage import io
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [40]:
# predictor_path = 'shape_predictor_81_face_landmarks.dat'


# def face_extract(path):

#     img=cv2.imread(path)

#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#     hog_face_detector = dlib.get_frontal_face_detector()

#     detector = dlib.get_frontal_face_detector()
#     predictor = dlib.shape_predictor(predictor_path)

#     dets = detector(img, 0)

#     facePoints = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 78, 74, 79, 73, 72, 80, 71, 70, 69, 68, 76, 75, 77, 0]

#     for k, d in enumerate(dets):
#         shape = predictor(img, d)
#         landmarks = np.matrix([[p.x, p.y] for p in shape.parts()])

#         mask = np.zeros(img.shape[:2], np.uint8)

#         face = np.array([ [shape.parts()[num].x, shape.parts()[num].y] for num in facePoints ])

#         cv2.drawContours(mask, [np.array(face)], -1, (255, 255, 255), -1, cv2.LINE_AA)

#         dst = cv2.bitwise_and(img, img, mask=mask)

#         plt.figure(figsize=(10, 10))
#         plt.imshow(dst)
#         return dst, img

In [53]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import cv2
warnings.filterwarnings('ignore')


def effecientDet_wrinkles(path):

  # IMAGE_PATHS = path

  PATH_TO_LABELS = '/content/traning_demo/annotations/label_map.pbtxt'

  category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


  print('Running inference... ', end='')

  # image = cv2.imread(IMAGE_PATHS)
  image = path
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_expanded = np.expand_dims(image_rgb, axis=0)


  input_tensor = tf.convert_to_tensor(image)

  input_tensor = input_tensor[tf.newaxis, ...]


  detections = detect_fn_wrinkles(input_tensor)

  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image_with_detections = image.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=0.4,
        agnostic_mode=False)

  return image_with_detections

def effecientDet_darkspots(path):

  # IMAGE_PATHS = path

  PATH_TO_LABELS = '/content/traning_demo/annotations/label_map.pbtxt'

  category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


  print('Running inference... ', end='')

  # image = cv2.imread(IMAGE_PATHS)
  image = path
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_expanded = np.expand_dims(image_rgb, axis=0)


  input_tensor = tf.convert_to_tensor(image)

  input_tensor = input_tensor[tf.newaxis, ...]


  detections = detect_fn_darkspots(input_tensor)

  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image_with_detections = image.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=0.4,
        agnostic_mode=False)

  return image_with_detections

In [ ]:
# img = effecientDet_wrinkles()

In [ ]:
# file_path = '/content/54196f0c2590c2a308ba2233b31552bf.jpg'
# img, orgImg = face_extract(file_path)
# plt.imshow(img)
# plt.show()


# img = effecientDet_wrinkles(img)
# cv2_imshow(img)
# img = effecientDet_darkspots(img)
# cv2_imshow(img)

# left_eye, right_eye, eye_points, img = eyes(img)
# img, percent = predict_puffyeyes(left_eye, right_eye, eye_points, img, orgImg)
# # plt.imshow(img)
# # plt.show()
# cv2.imwrite(file_path, img)
# print(percent)

In [41]:
model=load_model('efficientnet.h5')
eye_cascade=cv2.CascadeClassifier('haarcascade_eye.xml')

def eyes(path):
	# img=cv2.imread(path)
	img = path.copy()
	eye_rects=eye_cascade.detectMultiScale(img)
	eye_points=[]
	for x,y,w,h in eye_rects:
		eye_points.append([x,y,w,h])
	x1=eye_points[0][0]
	y1=eye_points[0][1]
	x2=eye_points[0][0]+eye_points[0][2]
	y2=eye_points[0][1]+eye_points[0][3]+30
	left_eye=img[y1:y2,x1:x2]
	left_eye=Image.fromarray(left_eye)
	left_eye=left_eye.resize((224,224))
	left_eye=asarray(left_eye)
	if len(eye_points)>1:
		x1=eye_points[1][0]
		y1=eye_points[1][1]
		x2=eye_points[1][0]+eye_points[1][2]  
		y2=eye_points[1][1]+eye_points[1][3]+30
		right_eye=img[y1:y2,x1:x2]
		right_eye=Image.fromarray(right_eye)
		right_eye=right_eye.resize((224,224))
		right_eye=asarray(right_eye)
	else:
		right_eye=0
	return left_eye, right_eye, eye_points, img

def predict_puffyeyes(left_eye,right_eye,l1,img, orgImg):
	test_image=np.expand_dims(left_eye,axis=0)
	result=model.predict(test_image)
	tempImg = orgImg.copy()
	if result[0][0]>result[0][1]:
		return orgImg, "No puffy eyes present"
	else:
		#print('left puffy eyes percentage',result[0][1]*100,'%')
		b=round(result[0][1]*100,2)
		x,y,w,h=l1[0]
		cv2.rectangle(tempImg, (x,y), (x+w,y+h+30), (0,255,255), 2)
		cv2.putText(tempImg,'Puffy eyes',(x,y+h+15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,25), 1, cv2.LINE_AA)
	if type(right_eye)!=int:
		test_image=np.expand_dims(right_eye,axis=0)
		result=model.predict(test_image)
		if result[0][0]>result[0][1]:
			return orgImg, "No puffy eyes present"
		else:
			#print('right puffy eyes percentage',result[0][1]*100,'%')
			a=round(result[0][1]*100,1)
			x,y,w,h=l1[1]
			cv2.rectangle(tempImg, (x,y), (x+w,y+h+30), (0,255,255), 2)
			cv2.putText(tempImg,'Puffy eyes',(x,y+h+15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,25), 1, cv2.LINE_AA)
			return tempImg, ""
	return orgImg, ""

In [ ]:
# img, orgImg = face_extract('/content/traning_demo/dataset/test/darkspots(41).jpg')
# left_eye, right_eye, eye_points, img = eyes(img)
# img, percent = predict_puffyeyes(left_eye, right_eye, eye_points, img, orgImg)
# img = effecientDet(img)
# print(percent)
# plt.imshow(img)

In [59]:
def load_img(path) :
  img=cv2.imread(path)
  return img

In [71]:

UPLOAD_FOLDER = 'static/uploads/'

app = Flask(__name__, template_folder='templates')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

run_with_ngrok(app)

ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_image():
	print("File uploaded")
	if 'file' not in request.files:
		return redirect(request.url)
	file = request.files['file']
	if file.filename == '':
		flash('No image selected for uploading')
		return redirect(request.url)
	if file and allowed_file(file.filename):
		filename = secure_filename(file.filename)
		

		file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
		file.save(file_path)
		print('upload_image filename: ' + filename)
		print(file_path)
		percent = ""
		try:
				# img, orgImg = face_extract(file_path)
				img = load_img(file_path)
				orgImg = img

				img = effecientDet_wrinkles(img)
				# cv2_imshow(img)
				img = effecientDet_darkspots(img)
				# cv2_imshow(img)
				orgImg = img.copy()
				left_eye, right_eye, eye_points, img = eyes(img)    
				img, percent = predict_puffyeyes(left_eye, right_eye, eye_points, img, img)
				# cv2_imshow(img)

				cv2.imwrite(file_path, img)
				print(percent)

				return render_template('result.html', filename=filename, eyes=percent)
		except :
				print("exception in eyes")
				cv2.imwrite(file_path, img)
				return render_template('result.html', filename=filename, eyes="No puffy eyes present")
	else:
		# flash('Allowed image types are -> png, jpg, jpeg, gif')
		return redirect(request.url)

@app.route('/display/<filename>')
def display_image(filename):
	print('display_image filename: ' + filename)
	return redirect(url_for('static', filename='uploads/' + filename), code=301)

if __name__ == "__main__":
	app.run()